In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
torch.cuda.empty_cache()
print(f"Using device: {device}")

Using device: cuda


In [2]:
def simulate_lorenz(initial_state, steps = 200, dt= 0.01, sigma = 10.0, rho = 28.0, beta = 8.0/3.0):
    def lorenz_system(state):
        x, y, z = state
        dxdt = sigma * (y - x)
        dydt = x * (rho - z) - y
        dzdt = x * y - beta * z
        return np.array([dxdt, dydt, dzdt])
    state = np.array(initial_state)
    trajectory = []
    for _ in range(steps):
        trajectory.append(state.copy())
        k1 = lorenz_system(state)
        k2 = lorenz_system(state + 0.5 * dt * k1)
        k3 = lorenz_system(state + 0.5 * dt * k2)
        k4 = lorenz_system(state + dt * k3)
        state += (dt / 6.0) * (k1 + 2*k2 + 2*k3 + k4)
    return np.array(trajectory)
        

In [3]:
def create_graph_lorenz(trajectory):
    edge_index = []
    for i in range(len(trajectory)-1):
        edge_index.append([i, i+1])
    edge_index = torch.tensor(edge_index, dtype = torch.long).t().contiguous()
    x = torch.tensor(trajectory, dtype = torch.float)

    data = Data(x=x, edge_index = edge_index)
    return data

In [4]:
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNN, self).__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)  
        self.fc3 = nn.Linear(hidden_dim, output_dim)

        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

        self.norm1 = nn.LayerNorm(hidden_dim)
        self.norm2 = nn.LayerNorm(output_dim)

    def create_full_graph(self, x):
        n = x.size(0)
        edge_index = torch.stack([
            torch.repeat_interleave(torch.arange(n), n),
            torch.tile(torch.arange(n), (n,))
        ]).to(x.device)
        return edge_index
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = x.float()
        edge_index = self.create_full_graph(x)
        
        # GNN path
        gnn_x = self.conv1(x, edge_index).relu()
        gnn_x = self.norm1(gnn_x)
        gnn_x = self.conv2(gnn_x, edge_index)
        gnn_x = self.norm2(gnn_x)

        # FC path - fixed the layer usage
        fc_x = self.fc1(x).relu()
        fc_x = self.fc2(fc_x).relu() 
        fc_x = self.fc3(fc_x)         

        return (gnn_x + fc_x)/2

In [5]:
class AdvancedKoopmanModel(torch.nn.Module):
    def __init__(self, input_dim, koopman_dim, hidden_dim=32):
        super(AdvancedKoopmanModel, self).__init__()
        self.encoder = GNN(input_dim, hidden_dim, koopman_dim)
        self.decoder = GNN(koopman_dim, hidden_dim, input_dim)
        init_matrix = torch.zeros(koopman_dim, koopman_dim)
        for i in range(0, koopman_dim-1, 2):
            init_matrix[i:i+2, i:i+2] = torch.tensor([[0., -1.], [1., 0.]])
        self.koopman_matrix = torch.nn.Parameter(init_matrix)
        
        self.register_buffer('running_mean', torch.zeros(input_dim))
        self.register_buffer('running_std', torch.ones(input_dim))
        self.register_buffer('num_batches_tracked', torch.tensor(0, dtype=torch.long))

    def update_statistics(self, x):
        if self.training:
            with torch.no_grad():
                batch_mean = x.mean(dim=0)
                batch_std = x.std(dim=0)
                
                if self.num_batches_tracked == 0:
                    self.running_mean = batch_mean
                    self.running_std = batch_std
                else:
                    momentum = 0.1
                    self.running_mean = (1 - momentum) * self.running_mean + momentum * batch_mean
                    self.running_std = (1 - momentum) * self.running_std + momentum * batch_std
                
                self.num_batches_tracked += 1

    def metric_loss(self, g, states, split_size=2):
        batch_size = g.size(0)
        permu = torch.randperm(batch_size)
        split_0 = permu[:batch_size//split_size]
        split_1 = permu[batch_size//split_size:2*(batch_size//split_size)]
        dist_g = torch.mean((g[split_0] - g[split_1]) ** 2, dim=1)
        dist_s = torch.mean((states[split_0] - states[split_1]) ** 2, dim=1)
        
        scaling_factor = 10.0
        return torch.abs(dist_g * scaling_factor - dist_s).mean()

    def system_identify(self, G, H, regularization=0.1):
        batch_size = G.size(0)
        I = torch.eye(self.koopman_matrix.size(0)).to(G.device)
        A = torch.matmul(H.transpose(1, 2), G) @ torch.inverse(
            torch.matmul(G.transpose(1, 2), G) + regularization * I
        )
        return A

    def forward(self, data):
        self.update_statistics(data.x)
        koopman_space = self.encoder(data)
        next_koopman_space = koopman_space @ self.koopman_matrix
        new_data = Data(x=next_koopman_space, edge_index=data.edge_index)
        decoded_state = self.decoder(new_data)
        
        return decoded_state

In [6]:
def advanced_loss(model, data, pred, lambda_metric=0.1, lambda_reg=0.01):
    recon_loss = F.mse_loss(pred, data.x)
    metric_loss = model.metric_loss(model.encoder(data), data.x)
    reg_loss = lambda_reg * torch.norm(model.koopman_matrix)
    
    return recon_loss + lambda_metric * metric_loss + reg_loss


In [7]:
def train_advanced_model(model, dataset, epochs=50, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    
    train_losses = []
    
    for epoch in range(epochs):
        epoch_loss = 0
        for data in dataset:
            optimizer.zero_grad()
            
            pred = model(data)
            
            loss = advanced_loss(model, data, pred)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
        
        avg_loss = epoch_loss / len(dataset)
        scheduler.step(avg_loss)
        train_losses.append(avg_loss)
        
        print(f"Epoch {epoch}, Loss: {avg_loss:.6f}")
    
    return train_losses

In [8]:
model = AdvancedKoopmanModel(input_dim=3, koopman_dim=16)

In [9]:
dataset = []
for _ in range(100): 
    initial_state = [
        np.random.normal(1.0, 0.2),
        np.random.normal(0.0, 0.2),
        np.random.normal(0.0, 0.2)
    ]
    trajectory = simulate_lorenz(initial_state, steps=1000, dt=0.01) 
    data = create_graph_lorenz(trajectory)
    dataset.append(data)

In [270]:
losses = train_advanced_model(model, dataset)

Epoch 0, Loss: 148.247553
Epoch 1, Loss: 2.935909
Epoch 2, Loss: 0.686900
Epoch 3, Loss: 0.423208
Epoch 4, Loss: 0.274925
Epoch 5, Loss: 0.232765
Epoch 6, Loss: 0.210570
Epoch 7, Loss: 0.201241
Epoch 8, Loss: 0.185007
Epoch 9, Loss: 0.172650
Epoch 10, Loss: 0.156881
Epoch 11, Loss: 0.129595
Epoch 12, Loss: 0.104744
Epoch 13, Loss: 0.092362
Epoch 14, Loss: 0.084623
Epoch 15, Loss: 0.084972
Epoch 16, Loss: 0.084832
Epoch 17, Loss: 0.081375
Epoch 18, Loss: 0.078147
Epoch 19, Loss: 0.076628
Epoch 20, Loss: 0.074552
Epoch 21, Loss: 0.073555
Epoch 22, Loss: 0.073847
Epoch 23, Loss: 0.072256
Epoch 24, Loss: 0.077164
Epoch 25, Loss: 0.071699
Epoch 26, Loss: 0.074641
Epoch 27, Loss: 0.075953
Epoch 28, Loss: 0.066800
Epoch 29, Loss: 0.074167
Epoch 30, Loss: 0.066932
Epoch 31, Loss: 0.071477
Epoch 32, Loss: 0.068727
Epoch 33, Loss: 0.065635
Epoch 34, Loss: 0.066251
Epoch 35, Loss: 0.066057
Epoch 36, Loss: 0.073897
Epoch 37, Loss: 0.062618
Epoch 38, Loss: 0.063868
Epoch 39, Loss: 0.066236
Epoch 40

In [18]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

In [19]:
save_folder = "lorenz-koopman-models"
os.makedirs(save_folder, exist_ok = True)

In [21]:
save_path = os.path.join(save_folder, "lorenz-koopman-model-3.5.pth")

In [275]:

torch.save(model.state_dict(), save_path)

print(f"Model saved to {save_path}")

Model saved to lorenz-koopman-models\lorenz-koopman-model-3.5.pth


In [25]:
model = AdvancedKoopmanModel(input_dim=3, koopman_dim=16).to(device)
model.load_state_dict(torch.load(save_path, weights_only=True))
model.eval()

AdvancedKoopmanModel(
  (encoder): GNN(
    (fc1): Linear(in_features=3, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=32, bias=True)
    (fc3): Linear(in_features=32, out_features=16, bias=True)
    (conv1): GCNConv(3, 32)
    (conv2): GCNConv(32, 16)
    (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  )
  (decoder): GNN(
    (fc1): Linear(in_features=16, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=32, bias=True)
    (fc3): Linear(in_features=32, out_features=3, bias=True)
    (conv1): GCNConv(16, 32)
    (conv2): GCNConv(32, 3)
    (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
  )
)

In [31]:
# Get the first trajectory from dataset
# test_data = dataset[74].to(device)

initial_state = [
    np.random.normal(1.0, 0.2),
    np.random.normal(0.0, 0.2),
    np.random.normal(0.0, 0.2)
]
trajectory = simulate_lorenz(initial_state, steps=6000, dt=0.01) 
test_data = create_graph_lorenz(trajectory).to(device)

# Get predictions
with torch.no_grad():
    predicted = model(test_data).cpu().numpy()
    actual = test_data.x.cpu().numpy()

# Create the visualization
fig = plt.figure(figsize=(15, 5))

# 3D trajectory plot
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot(actual[:, 0], actual[:, 1], actual[:, 2], 
         'b-', label='Actual', alpha=0.7)
ax1.plot(predicted[:, 0], predicted[:, 1], predicted[:, 2], 
         'r--', label='Predicted', alpha=0.7)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('3D Trajectory Comparison')
ax1.legend()

# Time series plot
ax2 = fig.add_subplot(122)
time = np.arange(len(actual))
ax2.plot(time, actual[:, 0], 'b-', label='Actual X', alpha=0.7)
ax2.plot(time, predicted[:, 0], 'r--', label='Predicted X', alpha=0.7)
ax2.set_xlabel('Time Step')
ax2.set_ylabel('X Coordinate')
ax2.set_title('X Coordinate Time Series')
ax2.legend()

plt.tight_layout()
plt.show()

# Print statistics
print("Actual trajectory range:", 
      np.min(actual, axis=0), 
      np.max(actual, axis=0))
print("Predicted trajectory range:", 
      np.min(predicted, axis=0), 
      np.max(predicted, axis=0))

# Calculate and print error metrics
mse = np.mean((actual - predicted) ** 2)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(actual - predicted))
print("\nError Metrics:")
print(f"MSE: {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"MAE: {mae:.6f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.29 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 5.51 GiB is allocated by PyTorch, and 2.21 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)